<a href="https://colab.research.google.com/github/AROY13-PRAETORIAN/StyleGAN_2_ADA/blob/main/StyleGAN2_ADA_AROY13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **STYLEGAN_2_ADA WITH CUSTOM DATASET** 

In [5]:
%tensorflow_version 1.x
from numpy import random
!nvidia-smi
import os
from google.colab import drive
from pathlib import Path
content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mon Oct 18 10:26:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#create directory
if os.path.isdir("/content/drive/My Drive/style_gan2_ada"):
    %cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"
else:
    #installing stylegan2-ada
    %cd "/content/drive/My Drive/"
    !mkdir style_gan2_ada
    %cd style_gan2_ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/style_gan2_ada/stylegan2-ada


In [7]:
%cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"
!git config --global user.name "AROY13-PRAETORIAN"
!git config --global user.email "abhigyan.roy2019@vitstudent.ac.in"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/My Drive/style_gan2_ada/stylegan2-ada


# **DATASET CONVERSION TO .tfrecords**

In [ ]:
#upload to drive and give its path to zip_path
zip_path = "/content/drive/MyDrive/Datasets/resized.zip"
!unzip {zip_path} -d /content/

In [ ]:
dataset_path = "/content/resized" #the dataset.zip will be unzipped in runtime so give path to that folder
dataset_name = "resized" #dataset name give same as folder unzipped in runtime

!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

# **TRAINING**

In [ ]:
!python train.py --help

In [ ]:
#same as the dataset name given earlier
dataset_name = "resized"
#how often samples and .pkl fines are generaed are determined by this
snapshot_count = 2
#mirror images left to right
mirrored = True
#mirror images top to bottom
mirroredY = False
#metric
metric_list = None
#augments
augs = "bgc"
#if you are running it for the first time give the path to ffhq(+resolution)
#if you want to use transfer learning then use a pretrained model uploaded to drive and give path to it or if you are resuming training give path to last .pkl file created in directory
resume_from = ""
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bgc"

# **THIS PART IS FOR IMAGE GENERATION OR LATENT SPACE WALK GENERATION FROM PRETRAINED MODEL**

**IMPORTANT : RUN THIS CELL**

In [9]:
%pip install opensimplex
output_path = drive_path / 'MyDrive' / 'stylegan2_ada_output'
output_path_images = output_path / 'images'
stylegan2_main_path = drive_path / 'MyDrive' / 'style_gan2_ada' / 'stylegan2-ada'
if not output_path.is_dir():
    %mkdir "{output_path}"
output_path_images = output_path / 'images'
if not output_path_images.is_dir():
    %mkdir "{output_path_images}"

# **GENERATE IMAGES**

In [5]:
seed_init = random.randint(10000)
nbr_images = 100

generation_from = '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00005-abstract_art_512-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000100.pkl' # path to final trained model

!python "{stylegan2_main_path / 'generate.py'}" generate-images \
    --outdir="{output_path_images}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3317760/45929032 bytes (7.2%)6750208/45929032 bytes (14.7%)10305536/45929032 bytes (22.4%)13729792/45929032 bytes (29.9%)17334272/45929032 bytes (37.7%)20684800/45929032 bytes (45.0%)24272896/45929032 bytes (52.8%)27901952/45929032 bytes (60.8%)31375360/45929032 bytes (68.3%)34824192/45929032 bytes (75.8%)38387712/45929032 bytes (83.6%)41885696/45929032 bytes (91.2%)

# **LATENT SPACE EXPLORATION**

In [10]:
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_output = output_path
explored_network =  "/content/drive/MyDrive/GANTRAINING/FreaGAN.pkl"  # path to final trained model

seeds = [random.randint(20000) for i in range(20)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "{stylegan2_main_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_output}" --trunc=0.7 --walk-type="{walk_types[2]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20} #generates 400 frames

13608,10142,14691,8626,7810
Base seeds: [13608, 10142, 14691, 8626, 7810]
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2490368/45929032 bytes (5.4%)6160384/45929032 bytes (13.4%)9863168/45929032 bytes (21.5%)13533184/45929032 bytes (29.5%)16965632/45929032 bytes (36.9%)20414464/45929032 bytes (44.4%)24068096/45929032 bytes (52.4%)27779072/45929032 bytes (60.5%)31326208/45929032 bytes (68.2%)34889728/45929032 bytes (76.0%)38510592/45929032 bytes (83.8%)

# **ESRGAN UPSCALING(OPTIONAL)**

In [11]:
#@title
frames = "/content/drive/MyDrive/stylegan2_ada_output/frames"
%cd {frames}
print("Time                     GPU        Used      Total")
!nvidia-smi --query-gpu=timestamp,name,memory.used,memory.free --format=csv,noheader
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi --query-gpu=timestamp,name,memory.used,memory.free --format=csv,noheader

/content/drive/MyDrive/stylegan2_ada_output/frames
Time                     GPU        Used      Total
2021/10/18 10:30:22.807, Tesla P100-PCIE-16GB, 0 MiB, 16280 MiB
2021/10/18 10:30:35.637, Tesla P100-PCIE-16GB, 0 MiB, 16280 MiB


In [12]:
#@title
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

fatal: destination path 'Real-ESRGAN' already exists and is not an empty directory.
/content/drive/My Drive/stylegan2_ada_output/frames/Real-ESRGAN
     |████████████████████████████████| 149 kB 5.2 MB/s 
     |████████████████████████████████| 5.8 MB 51.1 MB/s 
     |████████████████████████████████| 185 kB 46.4 MB/s 
  Created wheel for basicsr: filename=basicsr-1.3.4.4-py3-none-any.whl size=185577 sha256=f780becb3542d91c33e01b3e758be4937f4245ba4257111f8c3cf5b186bfbb15
  Stored in directory: /root/.cache/pip/wheels/ae/4c/74/8658a322336367833c17a3777611cb59a6a5dcfb904afc64cb
Successfully built basicsr
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 177 kB 55.5 MB/s 
  Created wheel for facexlib: filename=facexlib-0.2.1.0-py3-none-any.whl size=57092 sha256=86571840a4192da8209e935f9a9535978bf095c783b17187450eab42f7104aa5
  Stored in directory: /root/.cache/pip/wheels/e3/33/8d/70d77abe7eca95c8b0f32f12867234c45c1089db9db7102d03
  Created wh

In [13]:
#@title
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

In [14]:
#@title
%cd {frames}
final_output = [i for i in glob.glob("*.png")]
list_of_genrated_frames=[]
output = frames
for i in final_output:
  abc = output+"/" + i 
  list_of_genrated_frames.append(abc)
print(list_of_genrated_frames)

/content/drive/MyDrive/stylegan2_ada_output/frames
['/content/drive/MyDrive/stylegan2_ada_output/frames/frame00000.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00001.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00002.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00003.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00004.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00005.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00006.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00007.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00008.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00009.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00010.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00011.png', '/content/drive/MyDrive/stylegan2_ada_output/frames/frame00012.png', '/content/drive/MyDrive/stylegan2_ada_output/frames

In [15]:
move_for_upscaling = frames+"/"+"Real-ESRGAN/upload"
for i in list_of_genrated_frames:
  shutil.move(i, move_for_upscaling)

In [16]:
#@title
%cd Real-ESRGAN
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale 4 --outscale 3.5 --half --face_enhance

/content/drive/My Drive/stylegan2_ada_output/frames/Real-ESRGAN
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /usr/local/lib/python3.7/dist-packages/facexlib/weights/detection_Resnet50_Final.pth

100% 104M/104M [00:01<00:00, 101MB/s] 
Downloading: "https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth" to /usr/local/lib/python3.7/dist-packages/gfpgan/weights/GFPGANCleanv1-NoCE-C2.pth

100% 332M/332M [00:03<00:00, 101MB/s]
Testing 0 frame00000
inference_realesrgan.py:84: UserWarning: The input image is large, try X2 model for better performance.
  warnings.warn('The input image is large, try X2 model for better performance.')
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered inte

# **Generate your video again**

In [18]:
import cv2
import numpy as np
import glob
img_array = []
for filename in glob.glob('/content/drive/MyDrive/stylegan2_ada_output/frames/Real-ESRGAN/results/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
name = 'upscaled_video2' #@param {type:"string"} 
name = name+'.avi'
out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()